In [144]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [145]:
# Read in all the words
words = open('C:/Users/pchavan/The Bridge/MLStuff/makemore/names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [146]:
#Build the vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [147]:
#build the dataset

block_size = 3 # context lenght: how many characters do we tale to predict the next one

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] #crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])        #80%
Xdev, Ydev = build_dataset(words[n1:n2])    #10%
Xte, Yte = build_dataset(words[n2:])        #10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [148]:
#Utility tools that we'll use later when comparing manual gradients to PyTorch gradients 

def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exat: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [149]:
n_embd = 10 # The dimentionality of character embedding vectors
n_hidden = 64 # The number of neurons in the hidden layer of MLP

g = torch.Generator().manual_seed(2147483647)       # For reproducibility
C = torch.randn((vocab_size, n_embd),               generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden),   generator=g) * (5 / 3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden,                          generator=g) * 0.1      # b1 here is just for backprop practice; it's not required because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),            generator=g) * 0.1
b2 = torch.randn(vocab_size,                        generator=g) * 0.1
# Batchnorm Parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [150]:
batch_size = 32
n = batch_size # a shorter variable also, for conveniacnce

# Construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]   # Batch X, Y

In [151]:
#forward pass, "chunkated" into smaller steps that are possible to backward one at a time 

emb = C[Xb]                                                     # embed the characters ontp vecters 
embcat = emb.view(emb.shape[0], -1)                             # Concate the vectors
# Linear Layer 1
hprebn = embcat @ W1 + b1                                       # hidden layer preactivation
# Batchnorm layer
bnmeani = 1/n * hprebn.sum(0, keepdim = True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * bndiff2.sum(0, keepdim = True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-Linearity
h = torch.tanh(hpreact)                                         # Hidden layer
# Linear Layer 2 
logits = h @ W2 + b2                                            # Output Layer             
# Cross entropy loss (Same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim = True).values                            
norm_logits = logits - logit_maxes                              # normalizing Logits
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum ** -1                               
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward  pass
for p in parameters:
    p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv
          , norm_logits, logit_maxes, logits, h, hpreact, bnraw
          , bnvar, bnvar_inv, bndiff, bndiff2, hprebn, bnmeani
          , embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3547, grad_fn=<NegBackward0>)

## Becoming a backprop Ninja!

In [152]:
# Exercise 1: backprop through the whole thing manually, backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

In [211]:
dlogprob = torch.zeros(logprobs.shape)
dlogprob[range(n), Yb] = -1/n
cmp('logprobs', dlogprob, logprobs)

dprobs = dlogprob * 1 / probs
cmp('probs', dprobs, probs)

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

dcounts = dprobs * counts_sum_inv

dcounts_sum = dcounts_sum_inv * -1 * (counts_sum ** -2)
cmp('counts_sum', dcounts_sum, counts_sum)

dcounts += (torch.ones_like(counts) * dcounts_sum)
cmp('counts', dcounts, counts)

dnorm_logits = dcounts * norm_logits.exp()
cmp('norm_logits', dnorm_logits, norm_logits)

dlogits = dnorm_logits.clone()
dlogit_maxes = -dnorm_logits.sum(1, keepdim=True)
cmp('dlogit_maxes', dlogit_maxes, logit_maxes)

# tmp = torch.zeros_like(logits)
# tmp[range(n), (logits.max(1).indices)] = 1
# dlogits += dlogit_maxes * tmp

#OR

dlogits += dlogit_maxes * F.one_hot(logits.max(1).indices, num_classes=logits.shape[1])
cmp('dlogits', dlogits, logits)

dh = dnorm_logits @ W2.T
cmp('h', dh, h)

dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

db2 = dnorm_logits.sum(0)
cmp('b2', db2, b2)

# dhpreact = dh * (4 / (torch.exp(hpreact) + torch.exp(-hpreact))**2)
dhpreact = dh * (1.0 - h**2)
cmp('hpreact', dhpreact, hpreact)

dbngain = (dhpreact * bnraw).sum(0, keepdim = True)
dbnraw = dhpreact * bngain
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('dbngain', dbngain, bngain)
cmp('dbnraw', dbnraw, bnraw)
cmp('dbnbias', dbnbias, bnbias)

dbndiff = dbnraw * bnvar_inv
dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim = True)
# cmp('dbndiff', dbndiff, bndiff)
cmp('dbnvar_inv', dbnvar_inv, bnvar_inv)

dbnvar = dbnvar_inv * -0.5 * ((bnvar + 1e-5)**-1.5)
cmp('dbnvar', dbnvar, bnvar)

dbndiff2 = dbnvar * 1/(n-1) * torch.ones_like(bndiff2)
cmp('dbndiff2', dbndiff2, bndiff2)

dbndiff += dbndiff2 * 2 * bndiff
cmp('dbndiff', dbndiff, bndiff)

dhprebn = dbndiff.clone()
dbnmeani = -dbndiff.sum(0, keepdim=True)
cmp('dbnmeani', dbnmeani, bnmeani)

dhprebn += dbnmeani * 1/n * torch.ones_like(hprebn)
cmp('dhprebn', dhprebn, hprebn)

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
cmp('dembcat', dembcat, embcat)
cmp('dW1', dW1, W1)
cmp('db1', db1, b1)

demb = dembcat.view(emb.shape)
cmp('demb', demb, emb)

dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += demb[k,j]
cmp('dC', dC, C)

logprobs        | exat: True  | approximate: True  | maxdiff: 0.0
probs           | exat: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exat: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exat: True  | approximate: True  | maxdiff: 0.0
counts          | exat: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exat: True  | approximate: True  | maxdiff: 0.0
dlogit_maxes    | exat: True  | approximate: True  | maxdiff: 0.0
dlogits         | exat: True  | approximate: True  | maxdiff: 0.0
h               | exat: False | approximate: True  | maxdiff: 1.862645149230957e-09
W2              | exat: True  | approximate: True  | maxdiff: 0.0
b2              | exat: False | approximate: True  | maxdiff: 1.4901161193847656e-08
hpreact         | exat: False | approximate: True  | maxdiff: 1.3969838619232178e-09
dbngain         | exat: False | approximate: True  | maxdiff: 3.725290298461914e-09
dbnraw          | exat: False | approximate: True  | maxdiff: 1.3969

In [176]:
#emb = C[Xb]
demb.shape, C[Xb].shape

(torch.Size([32, 3, 10]), torch.Size([32, 3, 10]))

In [158]:
dC = torch.zeros_like(C)
dC[Xb] += demb

torch.Size([32, 3, 10])

In [ ]:
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += emb[k,j]

In [209]:
demb[0,1]

tensor([-1.9910e-03,  1.6615e-03,  2.4930e-03,  3.8594e-03, -2.7856e-05,
        -6.2349e-04,  1.5953e-04,  2.4422e-03,  3.3883e-03, -4.3675e-03],
       grad_fn=<SelectBackward0>)